In [258]:
# what genes interact with MEIS1 to contribute to 'restless leg syndrome'?
# does iron deficiency contributes to 'restless leg syndrome'?
# what variants of MEIS1 contribute to 'restless leg syndrome'?
# what drugs target iron deficiency?

# see how many ARAs can return MEIS1 with various identifiers - hopefully equivalent results
# HGNC:7000, OMIM:601739, NCBIGene:4211

# what if someone searches for protein interactions of MEIS1?
# n0 - MEIS1, n1 - category:['biolink:Protein']
# UniProtKB:O00470

# start with iron deficiency


import json
import requests
from gamma_viewer import GammaViewer
from IPython.display import display
from pprint import pprint

In [259]:
def trapi_query(json_str, url):
    return requests.post(url, 
                         json=json.loads(json_str))

In [260]:
def query_ara(aras, jstr):
    results = []
    count = 0
    for ara in aras:
        r = trapi_query(jstr,ara)
        j = r.json()
        edges = j['message']['knowledge_graph']['edges']
        
        for edge_key in edges.keys():
            edge = edges.get(edge_key)
            result = {'curie': edge.get('object'), 'predicate': edge.get('predicate'), 'pValue': edge.get('attributes')[0].get('value')}
            results.append(edge.get('subject') + " " + edge.get('predicate') + " " + edge.get('object'))
            count = count + 1
    return results, count;

In [261]:
jstr_n = """
{
  "message": {
    "query_graph": 
{
  "nodes": {
    "n0": {
      "id": "NCBIGene:4211",
      "category": ["biolink:Gene"]
    },
    "n1": {
      "category": [
        "biolink:Gene"
      ]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    }
  }
}
}
}

"""

# https://strider.renci.org/query strider
# https://evidara.healthdatascience.cloud/api/v1/query improvingAgent
# https://smart-api.info/query OpenAPI for NCATS Biomedical Translator
# https://explanatory-agent.azurewebsites.net/v1.0/query/ Explanatory Agent
# https://arax.ncats.io/api/arax/v1.0/query ARAX
# https://api.bte.ncats.io/v1/query BTE
# https://aragorn.renci.org/query ARAGORN



In [262]:
aras_smart = ['https://smart-api.info/query']

try:
    resultssm_n, countersm_n = query_ara(aras_smart, jstr_n)
    print ("count of results: %", countersm_n)
    pprint(resultssm_n)
except KeyError:
    print('fail, no results')
except:
    print('fail JSONDecode error')


fail JSONDecode error


In [263]:
aras_exp = ['https://explanatory-agent.azurewebsites.net/v1.0/query/']

try:
    resultse_n, countere_n = query_ara(aras_exp, jstr_n)
    print ("count of results: %", countere_n)
    pprint(resultse_n)
except KeyError:
    print('fail, no results')
except:
    raise


count of results: % 0
[]


In [249]:
aras_strider = ['https://strider.renci.org/query']

try:
    resultss_n, counters_n = query_ara(aras_strider, jstr_n)
    print ("count of results: %", counters_n)
    pprint(resultss_n)
except KeyError:
    print('fail, no results')
except:
    raise


count of results: % 85
['NCBIGene:4211 biolink:related_to NCBIGene:5431',
 'NCBIGene:4211 biolink:related_to NCBIGene:9070',
 'NCBIGene:4211 biolink:related_to NCBIGene:3214',
 'NCBIGene:4211 biolink:related_to NCBIGene:255626',
 'NCBIGene:4211 biolink:related_to NCBIGene:4211',
 'NCBIGene:4211 biolink:related_to NCBIGene:5080',
 'NCBIGene:4211 biolink:related_to NCBIGene:5437',
 'NCBIGene:4211 biolink:related_to NCBIGene:5434',
 'NCBIGene:4211 biolink:related_to NCBIGene:3198',
 'NCBIGene:4211 biolink:related_to NCBIGene:23512',
 'NCBIGene:4211 biolink:related_to NCBIGene:8348',
 'NCBIGene:4211 biolink:related_to NCBIGene:5439',
 'NCBIGene:4211 biolink:related_to NCBIGene:8334',
 'NCBIGene:4211 biolink:related_to NCBIGene:84824',
 'NCBIGene:4211 biolink:related_to NCBIGene:114781',
 'NCBIGene:4211 biolink:related_to NCBIGene:3235',
 'NCBIGene:4211 biolink:related_to NCBIGene:5090',
 'NCBIGene:4211 biolink:related_to NCBIGene:9611',
 'NCBIGene:4211 biolink:related_to NCBIGene:5438',
 '

In [264]:
aras_bte = ['https://api.bte.ncats.io/v1/query']

try:
    resultsb_n, counterb_n = query_ara(aras_bte, jstr_n)
    print ("count of results: %", counterb_n)
    pprint(resultsb_n)
except KeyError:
    print('fail, no results')
except:
    raise


count of results: % 231
['NCBIGene:4211 biolink:homologous_to MGI:104717',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:5087',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:3205',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:4212',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:4010',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:10586',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:4081',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:23373',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:1385',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:1387',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:3235',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:3227',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:158234',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:134553',
 'NCBIGene:4211 biolink:physically_interacts_with NCBIGene:31

In [246]:
aras_arax = ['https://arax.ncats.io/api/arax/v1.0/query']

try:
    resultsx_n, counterx_n = query_ara(aras_arax, jstr_n)
    print ("count of results: %", counterx_n)
    pprint(resultsx_n)
except KeyError:
    print('no results')
except:
    raise



count of results: % 241
['UniProtKB:O00470 biolink:coexists_with UniProtKB:P40424',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P17483',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P20719',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P28356',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P31268',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P31269',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P40424',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P40425',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P40426',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:P55347',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:Q13129',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:Q7Z417',
 'UniProtKB:O00470 biolink:physically_interacts_with UniProtKB:Q8NCA5',
 'UniProtKB:O00470 biolink:physically_interacts_with

In [265]:
aras_aragorn = ['https://aragorn.renci.org/query']

try:
    resultsag_n, counterag_n = query_ara(aras_aragorn, jstr_n)
    print ("count of results: %", counterag_n)
    pprint(resultsag_n)
except KeyError:
    print('no results')
except:
    raise


count of results: % 121
['NCBIGene:4211 biolink:related_to NCBIGene:5431',
 'NCBIGene:4211 biolink:related_to NCBIGene:9070',
 'NCBIGene:4211 biolink:related_to NCBIGene:3214',
 'NCBIGene:4211 biolink:related_to NCBIGene:255626',
 'NCBIGene:4211 biolink:related_to NCBIGene:4211',
 'NCBIGene:4211 biolink:related_to NCBIGene:5080',
 'NCBIGene:4211 biolink:related_to NCBIGene:5437',
 'NCBIGene:4211 biolink:related_to NCBIGene:5434',
 'NCBIGene:4211 biolink:related_to NCBIGene:3198',
 'NCBIGene:4211 biolink:related_to NCBIGene:23512',
 'NCBIGene:4211 biolink:related_to NCBIGene:8348',
 'NCBIGene:4211 biolink:related_to NCBIGene:5439',
 'NCBIGene:4211 biolink:related_to NCBIGene:8334',
 'NCBIGene:4211 biolink:related_to NCBIGene:84824',
 'NCBIGene:4211 biolink:related_to NCBIGene:114781',
 'NCBIGene:4211 biolink:related_to NCBIGene:3235',
 'NCBIGene:4211 biolink:related_to NCBIGene:5090',
 'NCBIGene:4211 biolink:related_to NCBIGene:9611',
 'NCBIGene:4211 biolink:related_to NCBIGene:5438',
 

In [266]:
aras_improving = ['https://evidara.healthdatascience.cloud/api/v1/query']
try:
    resultsi_n, countersi_n = query_ara(aras_improving, jstr_n)
    print ("count of results: %", countersi_n)
    pprint(resultsi_n)
except KeyError:
    print('no results')
except:
    raise


count of results: % 4
['NCBIGene:4211 biolink:positively_regulates_entity_to_entity NCBIGene:993',
 'NCBIGene:4211 biolink:positively_regulates_entity_to_entity NCBIGene:7855',
 'NCBIGene:4211 biolink:positively_regulates_entity_to_entity NCBIGene:5728',
 'NCBIGene:4211 biolink:positively_regulates_entity_to_entity NCBIGene:6389']


In [267]:
jstr_h = """
{
  "message": {
    "query_graph": 
{
  "nodes": {
    "n0": {
      "id": "HGNC:7000",
      "category": ["biolink:Gene"]
    },
    "n1": {
      "category": [
        "biolink:Gene"
      ]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    }
  }
}
}
}

"""

# https://strider.renci.org/query strider
# https://evidara.healthdatascience.cloud/api/v1/query improvingAgent
# https://smart-api.info/query OpenAPI for NCATS Biomedical Translator
# https://explanatory-agent.azurewebsites.net/v1.0/query/ Explanatory Agent
# https://arax.ncats.io/api/arax/v1.0/query ARAX
# https://api.bte.ncats.io/v1/query BTE
# https://aragorn.renci.org/query ARAGORN



In [268]:
aras_smart = ['https://smart-api.info/query']

try:
    resultssm_h, countersm_h = query_ara(aras_smart, jstr_h)
    print ("count of results: %", countersm_h)
    pprint(resultssm_h)
except KeyError:
    print('fail, no results')
except:
    print('fail JSONDecode error')


fail JSONDecode error


In [269]:
aras_exp = ['https://explanatory-agent.azurewebsites.net/v1.0/query/']

try:
    resultse_h, countere_h = query_ara(aras_exp, jstr_h)
    print ("count of results: %", countere_h)
    pprint(resultse_h)
except KeyError:
    print('fail, no results')
except:
    raise


count of results: % 0
[]


In [ ]:
aras_strider = ['https://strider.renci.org/query']

try:
    resultss_h, counters_h = query_ara(aras_strider, jstr_h)
    print ("count of results: %", counters_h)
    pprint(resultss_h)
except KeyError:
    print('fail, no results')
except:
    raise


In [ ]:
aras_bte = ['https://api.bte.ncats.io/v1/query']

try:
    resultsb_h, counterb_h = query_ara(aras_bte, jstr_h)
    print ("count of results: %", counterb_h)
    pprint(resultsb_h)
except KeyError:
    print('fail, no results')
except:
    raise


In [ ]:
aras_arax = ['https://arax.ncats.io/api/arax/v1.0/query']

try:
    resultsx_h, counterx_h = query_ara(aras_arax, jstr_h)
    print ("count of results: %", counterx_h)
    pprint(resultsx_h)
except KeyError:
    print('no results')
except:
    raise



In [ ]:
aras_aragorn = ['https://aragorn.renci.org/query']

try:
    resultsag_h, counterag_h = query_ara(aras_aragorn, jstr_h)
    print ("count of results: %", counterag_h)
    pprint(resultsag_h)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_improving = ['https://evidara.healthdatascience.cloud/api/v1/query']
try:
    resultsi_h, countersi_h = query_ara(aras_improving, jstr_h)
    print ("count of results: %", countersi_h)
    pprint(resultsi_h)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
jstr_o = """
{
  "message": {
    "query_graph": 
{
  "nodes": {
    "n0": {
      "id": "OMIM:601739",
      "category": ["biolink:Gene"]
    },
    "n1": {
      "category": [
        "biolink:Gene"
      ]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    }
  }
}
}
}

"""

# https://strider.renci.org/query strider
# https://evidara.healthdatascience.cloud/api/v1/query improvingAgent
# https://smart-api.info/query OpenAPI for NCATS Biomedical Translator
# https://explanatory-agent.azurewebsites.net/v1.0/query/ Explanatory Agent
# https://arax.ncats.io/api/arax/v1.0/query ARAX
# https://api.bte.ncats.io/v1/query BTE
# https://aragorn.renci.org/query ARAGORN



In [ ]:
aras_smart = ['https://smart-api.info/query']

try:
    resultssm_o, countersm_o = query_ara(aras_smart, jstr_o)
    print ("count of results: %", countersm_o)
    pprint(resultssm_o)
except KeyError:
    print('fail, no results')
except:
    print('fail JSONDecode error')


In [ ]:
aras_exp = ['https://explanatory-agent.azurewebsites.net/v1.0/query/']

try:
    resultse_o, countere_o = query_ara(aras_exp, jstr_o)
    print ("count of results: %", countere_o)
    pprint(resultse_o)
except KeyError:
    print('fail, no results')
except:
    raise


In [ ]:
aras_strider = ['https://strider.renci.org/query']

try:
    resultss_o, counters_o = query_ara(aras_strider, jstr_o)
    print ("count of results: %", counters_o)
    pprint(resultss_o)
except KeyError:
    print('fail, no results')
except:
    raise


In [ ]:
aras_bte = ['https://api.bte.ncats.io/v1/query']

try:
    resultsb_o, counterb_o = query_ara(aras_bte, jstr_o)
    print ("count of results: %", counterb_o)
    pprint(resultsb_o)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_arax = ['https://arax.ncats.io/api/arax/v1.0/query']
try:
    resultsx_o, counterx_o = query_ara(aras_arax, jstr_o)
    print ("count of results: %", counterx_o)
    pprint(resultsx_o)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_aragorn = ['https://aragorn.renci.org/query']

try:
    resultsag_o, counterag_o = query_ara(aras_aragorn, jstr_h)
    print ("count of results: %", counterag_o)
    pprint(resultsag_o)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_improving = ['https://evidara.healthdatascience.cloud/api/v1/query']

try:
    resultsi_o, countersi_o = query_ara(aras_improving, jstr_o)
    print ("count of results: %", countersi_o)
    pprint(resultsi_o)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
jstr_ug = """
{
  "message": {
    "query_graph": 
{
  "nodes": {
    "n0": {
      "id": "UniProtKB:O00470",
      "category": ["biolink:Gene"]
    },
    "n1": {
      "category": [
        "biolink:Gene"
      ]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    }
  }
}
}
}

"""

# https://strider.renci.org/query strider
# https://evidara.healthdatascience.cloud/api/v1/query improvingAgent
# https://smart-api.info/query OpenAPI for NCATS Biomedical Translator
# https://explanatory-agent.azurewebsites.net/v1.0/query/ Explanatory Agent
# https://arax.ncats.io/api/arax/v1.0/query ARAX
# https://api.bte.ncats.io/v1/query BTE
# https://aragorn.renci.org/query ARAGORN



In [ ]:
aras_bte = ['https://api.bte.ncats.io/v1/query']

try:
    resultsb_u, counterb_u = query_ara(aras_bte, jstr_ug)
    print ("count of results: %", counterb_u)
    pprint(resultsb_u)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_arax = ['https://arax.ncats.io/api/arax/v1.0/query']

try:
    resultsx_u, countersx_u = query_ara(aras_arax, jstr_ug)
    print ("count of results: %", countersx_u)
    pprint(resultsx_u)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_aragorn = ['https://aragorn.renci.org/query']

try:
    resultsag_u, counterag_u = query_ara(aras_aragorn, jstr_ug)
    print ("count of results: %", counterag_u)
    pprint(resultsag_u)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_improving = ['https://evidara.healthdatascience.cloud/api/v1/query']

try:
    resultsi_u, counteri_u = query_ara(aras_improving, jstr_ug)
    print ("count of results: %", counteri_u)
    pprint(resultsi_u)
except KeyError:
    print('no results')
except:
    raise



In [ ]:
jstr_up = """
{
  "message": {
    "query_graph": 
{
  "nodes": {
    "n0": {
      "id": "UniProtKB:O00470",
      "category": ["biolink:Protein"]
    },
    "n1": {
      "category": [
        "biolink:Protein"
      ]
    }
  },
  "edges": {
    "e0": {
      "subject": "n0",
      "object": "n1"
    }
  }
}
}
}

"""

# https://strider.renci.org/query strider
# https://evidara.healthdatascience.cloud/api/v1/query improvingAgent
# https://smart-api.info/query OpenAPI for NCATS Biomedical Translator
# https://explanatory-agent.azurewebsites.net/v1.0/query/ Explanatory Agent
# https://arax.ncats.io/api/arax/v1.0/query ARAX
# https://api.bte.ncats.io/v1/query BTE
# https://aragorn.renci.org/query ARAGORN



In [ ]:
aras_bte = ['https://api.bte.ncats.io/v1/query']

try:
    resultsb_up, counterb_up = query_ara(aras_bte, jstr_up)
    print ("count of results: %", counterb_up)
    pprint(resultsb_up)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_arax = ['https://arax.ncats.io/api/arax/v1.0/query']

try:
    resultsx_u, countersx_u = query_ara(aras_arax, jstr_up)
    print ("count of results: %", countersx_u)
    pprint(resultsx_u)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_aragorn = ['https://aragorn.renci.org/query']

try:
    resultsag_up, counterag_up = query_ara(aras_aragorn, jstr_up)
    print ("count of results: %", counterag_up)
    pprint(resultsag_up)
except KeyError:
    print('no results')
except:
    raise


In [ ]:
aras_improving = ['https://evidara.healthdatascience.cloud/api/v1/query']

try:
    resultsi_up, counteri_up = query_ara(aras_improving, jstr_up)
    print ("count of results: %", counteri_up)
    pprint(resultsi_up)
except KeyError:
    print('no results')
except:
    raise

